In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

base_df = pd.read_csv("/home/samir/Desktop/rudraAnalytics/sub_projects/churn/data/data.csv")
pd.set_option("display.max_columns", None)

In [5]:
df = base_df.copy()

In [6]:
# from prepare import screen_data
# screen_data(df=df, skip=['customerID'])

In [7]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)

In [8]:

def one_hot_encode(df, encode_set=[], dont_encode=[]):
    encoded_df = df.copy()
    for column in encode_set:
        if column in dont_encode:
            continue
        # print(len(df[column].unique()))
        if df[column].dtype == 'object':
            if len(df[column].unique()) == 2:
                # Binary encoding (0 and 1)
                encoded_df[column] = pd.get_dummies(df[column], drop_first=True)
            else:
                # One-hot encoding and using 0 and 1 instead of True and False
                one_hot_encoded = pd.get_dummies(df[column], prefix=column, drop_first=False)
                one_hot_encoded.columns = [f"{column}{i+1}" for i in range(one_hot_encoded.shape[1])]
                encoded_df = pd.concat([encoded_df, one_hot_encoded], axis=1)
                encoded_df.drop(column, axis=1, inplace=True)
                # print(encoded_df)

    return encoded_df

In [9]:
features = [
    'customerID',
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'tenure',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
    'MonthlyCharges',
    'TotalCharges',
    'Churn'
]
dont_label = ['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']

encoded_df = one_hot_encode(df.drop('customerID', axis=1), features, dont_label)
encoded_df['TotalCharges'].fillna(encoded_df['TotalCharges'].mean(), inplace=True)

In [12]:
X = encoded_df.drop('Churn', axis=1)
y = encoded_df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred_dl = (model.predict(X_test) > 0.5).astype(int)

# Evaluate the model
accuracy_dl = accuracy_score(y_test, y_pred_dl)
print(f"Deep Learning Test Accuracy: {accuracy_dl * 100:.2f}%")

Epoch 1/10
177/177 [==============================] - 1s 2ms/step - loss: 0.5599 - accuracy: 0.6999 - val_loss: 0.4310 - val_accuracy: 0.8027
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4374 - accuracy: 0.7906 - val_loss: 0.4160 - val_accuracy: 0.8048
Epoch 3/10
177/177 [==============================] - 0s 1ms/step - loss: 0.4269 - accuracy: 0.7971 - val_loss: 0.4081 - val_accuracy: 0.8133
Epoch 4/10
177/177 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.7987 - val_loss: 0.4047 - val_accuracy: 0.8119
Epoch 5/10
177/177 [==============================] - 0s 1ms/step - loss: 0.4182 - accuracy: 0.8003 - val_loss: 0.4034 - val_accuracy: 0.8162
Epoch 6/10
177/177 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.8024 - val_loss: 0.4028 - val_accuracy: 0.8119
Epoch 7/10
177/177 [==============================] - 0s 2ms/step - loss: 0.4137 - accuracy: 0.8051 - val_loss: 0.4033 - val_accuracy: 0.8126
Epoch 

Deep Learning Test Accuracy: 81.48%